In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.7 MB/s eta 0:00:00


In [2]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving Horizon Year 2050 -  Jn 4 Mitigation - AM Peak.pdf to Horizon Year 2050 -  Jn 4 Mitigation - AM Peak.pdf


In [3]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS'] )

In [6]:
for filename in uploaded.keys():
    all_text = []
    with pdfplumber.open(filename) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:  # Ensure the page contains text
                all_text.append(f"Page {page_num}:\n{page_text}\n")

    # Join the text of all pages into a single string, separating by newlines
    full_text = "\n".join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)
    #match = re.match(r"(\S+ \d+)\s(.+)", data[2])
    match = re.match(r"([\w\s]+?\d+)[\s:-]+(.+)", data[2])
    if match:
        node = match.group(1)  # Extract "Jn X"
        Scenario = match.group(2)
    data_string = "\n".join(data)
    data_string = re.sub(r"\nMovement", r"\n Movement", data_string)
    data_string = re.sub(r"\nIntersection", r"\n Intersection", data_string)
    pattern = r"Movement (.*?)Critical Lane Group"
    match = re.search(pattern, data_string, re.DOTALL)

    if match:
        Movemt = match.group(1).strip()  # Extracted content
    pattern = r"Intersection Summary(.*)"
    match = re.search(pattern, Movemt, re.DOTALL)

    if match:
        extracted_data = match.group(1).strip()

    pattern = r"HCM 2000 Control Delay (\S*) .*?Level of Service (\S*).*?Volume to Capacity ratio (\S*)"

    match = re.search(pattern, extracted_data, re.DOTALL)

    if match:
        control_delay, level_of_service, v_c_ratio = match.groups()
    new_row = {
        'Scenario': Scenario,
        'Node': node,
        'V/C':v_c_ratio,
        'Delay': control_delay,
        'LOS': level_of_service
    }

    # Use pd.concat to add the new row
    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
Output.to_excel('Synchro_Report.xlsx', index=False)